In [1]:
from scrapper.extraction.extraction import ExtractionStrategyBase, LLMExtractionStrategyIMAGE, LLMExtractionStrategyHTML
from linkedin.models import JobDescription

import os

from litellm import acompletion, completion
import json
from pathlib import Path
from pydantic import BaseModel, ValidationError
from scrapper.core.page_output import PageResponse
import litellm

from markitdown import MarkItDown


In [2]:
from scrapper.utils import read_json, save_json

In [3]:
filepath ="/home/t/atest/use_computer/saved_content/2024-12-15 15:15:17.037775/page_response-2024-12-15 15:15:17.103937.json"

In [4]:
a = read_json(filepath)

Dict/List length: 9


In [5]:
print(a.keys())

dict_keys(['screenshot_path', 'url', 'kind', 'extracted_data', 'text', 'html', 'markdown', 'clean_html', 'clean_html2'])


In [6]:
a

{'screenshot_path': '/home/t/atest/use_computer/saved_content/screenshots/0d72d9b0-5515-4f2d-b256-83a79d4616d3.png',
 'url': 'https://www.linkedin.com/jobs/view/4097738846/',
 'kind': 'LinkedInCategory.JOB_POSTING',
 'extracted_data': None,
 'text': "0 notifications total\nSkip to search\nSkip to main content\nKeyboard shortcuts\nClose jump menu\nnew feed updates notifications\nHome\nMy Network\nJobs\nMessaging\n2\n2 new notifications\nNotifications\nMe\nFor Business\nTry Premium for ₹0\nArtificial Intelligence Engineer\nArtificial Intelligence Engineer\nSecuritic.AI · India (Remote)\nShow more options\nSave\nSave Artificial Intelligence Engineer at Securitic.AI\nApply\nSecuritic.AI\nShare\nShow more options\nArtificial Intelligence Engineer\nIndia · 2 days ago · Over 100 people clicked apply\nRemote\nMatches your job preferences, workplace type is Remote.\n  Full-time\nMatches your job preferences, job type is Full-time.\n10 of 10 skills match your profile - you may be a good fit\nCur

In [7]:
rr = PageResponse(**a)

In [8]:
rr.screenshot_path

'/home/t/atest/use_computer/saved_content/screenshots/0d72d9b0-5515-4f2d-b256-83a79d4616d3.png'

In [9]:
Path('markdown.md').write_text(rr.markdown)

499527

In [11]:
Path('some_html.html').write_text(rr.html)

1292048

In [13]:
aa = MarkItDown().convert_local(path='some_html.html')

In [16]:
Path('New_markdown.md').write_text(aa.text_content)

497662

In [10]:
os.path.exists(rr.screenshot_path)

True

In [11]:
from bs4 import BeautifulSoup
import re

def clean_html(html_str):
    """
    Removes unnecessary HTML elements and attributes, keeping visual/text content.

    Args:
      html_str: A string of HTML.

    Returns:
      A string of cleaned HTML.
    """

    soup = BeautifulSoup(html_str, 'html.parser')

    # Remove comments
    for comment in soup.find_all(string=lambda text: isinstance(text, str) and '<!--' in text):
      comment.extract()

    # Remove script tags
    for script in soup.find_all('script'):
        script.extract()

    # Remove style tags
    for style in soup.find_all('style'):
        style.extract()

    # Remove noscript tags
    for noscript in soup.find_all('noscript'):
        noscript.extract()

    # Remove meta tags
    for meta in soup.find_all('meta'):
        meta.extract()

    # # Remove link tags (mostly for CSS, etc.)
    # for link in soup.find_all('link'):
    #   link.extract()

    # Remove img tags
    for img in soup.find_all('img'):
        img.extract()

    # Remove anchor tags
    for a in soup.find_all('a'):
        a.extract()

    # # Remove buttons
    # for button in soup.find_all('button'):
    #   button.extract()

    # Remove forms
    for form in soup.find_all('form'):
      form.extract()

    # Remove elements with specific classes and ids
    elements_to_remove = [
        "header", "footer", "navigation", "sidebar",
        "ad", "banner", "cookie-banner", "modal", "nav"
    ]
    for tag in soup.find_all(True, class_=lambda x: x in elements_to_remove if x else False) :
         tag.extract()
    for tag in soup.find_all(True, id=lambda x: x in elements_to_remove if x else False) :
         tag.extract()

    # Remove code tags
    for code in soup.find_all('code'):
          code.extract()
          
    # Remove style attributes
    for tag in soup.find_all(True):
        if 'style' in tag.attrs:
          del tag.attrs['style']

    # Remove class and id attributes (most)
    for tag in soup.find_all(True):
      if "class" in tag.attrs:
          del tag.attrs["class"]
      if "id" in tag.attrs:
          del tag.attrs["id"]

    # Remove empty tags (excluding br and hr)
    for tag in soup.find_all(True):
        if not tag.contents and tag.name not in ['br', 'hr']:
            tag.extract()

    # Return the cleaned HTML
    return str(soup)


In [11]:
a = cleaned_html = clean_html(rr.html)
# print(cleaned_html)

In [ ]:
Path('clean_html.html').write_text(a)

In [ ]:
x = '\n{"ramesh":"batsman"}'
x = json.loads(x)
x

In [14]:
from typing import Literal, Type, Callable, Optional, Any, List, Dict
from scrapper.extraction.extraction_utils import parse_image, get_dict

import logging

logger=logging.getLogger('me')

In [15]:
class LLMCombinedExtractionStrategy(ExtractionStrategyBase):
    """
    A class for extracting data using both HTML and image data with cross-referencing
    within a single LLM prompt.
    """

    def __init__(self, model: str, extraction_model: Type[BaseModel], api_key: str = None,
                 fallbacks=None, clean_html_func: Callable[[str], str] = clean_html, *args, **kwargs):
        """
        Initializes the LLMCombinedExtractionStrategy object.

        Args:
            model (str): The name of the language model to use.
            extraction_model (Type[BaseModel]): The Pydantic model class for the extracted data.
            api_key (str, optional): The API key for the language model. Defaults to None.
            fallbacks (list, optional): List of fallback models to use if the primary model fails. Defaults to None.
            clean_html_func (Callable, optional): A function to clean the HTML before sending to llm, (html files are big, should remove unnecessary elements not related to data extraction))
            verbose (bool, optional): Enable verbose output. Defaults to False.
            validate_json (bool, optional): Enable JSON schema validation. Defaults to True.
        """
        self.model = model
        self.__api_key = api_key
        self.extraction_model = extraction_model
        self.fallbacks = fallbacks
        self.clean_html_func = clean_html_func 
        self.verbose = kwargs.get('verbose', False)
        self.validate_json = kwargs.get('validate_json', True)
        self.model_schema = self.extraction_model.model_json_schema()
        self._setup_litellm(kwargs)


        self.extraction_prompt = """
You are given a screenshot of a webite and also the corresponding HTML, you have to extract various data fields mentioned below. 
look at both the image and the html and if you see the same information in both places use that else resolve the conflicts. 
With given visual and texual context, extract the requested field and only return a valid json object.
no explaination or anything is needed, pure json with data fields.

==FIELDS TO EXTRACT==
{fields_to_extract}
=====================

===HTML=======
{html_str}
=============
"""
    def _setup_litellm(self, kwargs):
        """Sets up litellm library settings."""
        litellm.enable_json_schema_validation = self.validate_json
        litellm.set_verbose = self.verbose

    def _preparation(self, page_response: PageResponse, *args, **kwargs):
          """Prepares the message and response format for the LLM."""
          html = self.clean_html_func(page_response.html) if self.clean_html_func else page_response.html

          prompt = self.extraction_prompt.format(fields_to_extract=self.model_schema, html_str=html)
          messages = [parse_image(image=page_response.screenshot_path, message=prompt)]
          response_format = {"type": "json_schema", "strict": True}
          return messages, response_format

    def parse_output_to_model(self, x: str) -> Optional[Type[BaseModel]]:
        """Parses the LLM output into a Pydantic model."""
        if x is None:
            return None

        x = get_dict(x)
        try:
            return self.extraction_model(**x)
        except ValidationError as e:
            logging.error(f"Validation error: {e}", exc_info=True)
            raise

    def extract(self, page_response: PageResponse, *args, **kwargs) -> PageResponse:
        """Extracts data by combining HTML and image analysis within a single prompt."""
        messages, response_format = self._preparation(page_response=page_response, *args, **kwargs)
        try:
            response = completion(model=self.model, messages=messages, api_key=self.__api_key,
                                response_format=response_format, fallbacks=self.fallbacks, stream=False, **kwargs)
        except Exception as e:
            logging.error(f"Extraction failed: {e}", exc_info=True)
            response = None

        if hasattr(response, 'choices'):
            response = response.choices[0].message.content
            page_response.extracted_data = self.parse_output_to_model(response)
        return page_response

    async def aextract(self, page_response: PageResponse, *args, **kwargs) -> PageResponse:
        """Asynchronously extracts data by combining HTML and image analysis within a single prompt."""
        messages, response_format = self._preparation(page_response=page_response, *args, **kwargs)
        try:
            response = await acompletion(model=self.model, messages=messages, api_key=self.__api_key,
                                     response_format=response_format, fallbacks=self.fallbacks, stream=False, **kwargs)
        except Exception as e:
            logging.error(f"Async extraction failed: {e}", exc_info=True)
            response = None
        if hasattr(response, 'choices'):
            response = response.choices[0].message.content
            page_response.extracted_data = self.parse_output_to_model(response)
        return page_response

In [16]:
llm = LLMCombinedExtractionStrategy(
    model='gemini/gemini-2.0-flash-exp', 
    # model='mistral/pixtral-large-latest',
    # model='mistral/pixtral-12b-2409',
    # model='mistral/pixtral-large-latest',
    # model='mistral/pixtral-large-latest',
    extraction_model=JobDescription, 
    response_type='json_object',
    validate_json=True,
    )

In [17]:
litellm.set_verbose=False

In [18]:
# llm = LLMExtractionStrategyHTML(
#     model='groq/llama-3.3-70b-versatile', 
#     extraction_model=JobDescription, 
#     litellm_info="EXCEPTION",
#     validate_json=True,
    
#     )

# llm = LLMExtractionStrategyIMAGE(
#     model='groq/llama-3.2-11b-vision-preview', 
#     extraction_model=JobDescription, 
#     response_type='json_schema',
#     validate_json=True,
#     )

In [19]:
# a,x=llm._preparation(rr)

In [20]:
# type(a[0]), a[0]

In [ ]:
f = llm.extract(rr)

In [29]:
dd = f.extracted_data
if isinstance(dd, BaseModel):
    dd = dd.model_dump()
elif isinstance(dd,str):
    dd = dd

In [30]:
dd

{'verified': True,
 'job_id': None,
 'job_title': 'Artificial Intelligence Engineer',
 'job_link': None,
 'location': 'India (Remote)',
 'salary': None,
 'on_site_or_remote': ['Remote'],
 'job_status': None,
 'posted_time': '2 days ago',
 'reply_time': None,
 'easy_apply': False,
 'promoted': False,
 'job_level': None,
 'job_type': ['Full-time'],
 'alumni_works_here': None,
 'people_engaged': {'clicked': None,
  'viewed': None,
  'applied': 100,
  'other': None},
 'skill_match': '10 of 10',
 'job_description': 'Company: Securitic.AI Location: Remote (India) Job Type: Full-time About the Job: Join our team as an innovative AI Engineer, where you will design and implement machine learning models and AI solutions. Your expertise will shape the future of intelligent systems for our organization. Skills Required: Machine Learning & Deep Learning (TensorFlow, PyTorch) Python Data Preprocessing & Feature Engineering Cloud AI Services (AWS Sagemaker, Google AI) Natural Language Processing (NLP

True


In [ ]:
f.extracted_data

In [ ]:
for i,j in dd.items():
    print(i, ": ", j)

In [ ]:
f.extracted_data[:40], f.extracted_data[-40:]

In [ ]:
ass = get_dict(f.extracted_data)
type(ass)

In [60]:
ss = f.extracted_data.lstrip("```json\n").rstrip('\n```\n')

In [ ]:
ss[:40], ss[-40:]

In [ ]:
ss

In [65]:
sd = JobDescription(**ss)

In [ ]:
print(sd)

In [38]:
def counterr(x)->int:
    return len(x.split())

In [ ]:
print('url: ',counterr(rr.url))
print('html: ', counterr(rr.html))
print('clean: ',counterr(rr.clean_html))
print('clean2: ',counterr(rr.clean_html2))
print('text: ', counterr(rr.text))

In [ ]:
rr.url

In [ ]:
f.extracted_data

In [18]:
import litellm


In [ ]:
litellm.set_verbose=None
a= completion(model='mistral/open-mistral-7b', messages=[{'role':'user', 'content':'who is shiv'}])

In [ ]:
a.choices[0].message.content

In [21]:
import base64
import os

def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None


In [ ]:
rr.screenshot_path

In [26]:
# image_path = '/home/t/images/toraniofficial/toraniofficial-26082024-0079.jpg'
image_path = rr.screenshot_path
base64_image = encode_image(image_path)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What's in this image?"
            },
            {
                "type": "image_url",
                "image_url": f"data:image/jpeg;base64,{base64_image}" 
            }
        ]
    }
]

# model='mistral/pixtral-large-latest'
model = 'mistral/pixtral-12b-2409'

a= completion(model=model, messages=messages)

In [ ]:
print(a.choices[0].message.content)

In [1]:
from linkedin.items.linkedin_objects import get_search_url, LinkedInCategory

In [2]:
a = get_search_url('Job')

In [3]:

a

SearchLinks(url='https://www.linkedin.com/jobs/search/', kind=<LinkedInCategory.JOB: 'Job'>)

In [4]:
aa = LinkedInCategory("job")
aa

ValueError: 'job' is not a valid LinkedInCategory

In [5]:
LinkedInCategory('NON')

ValueError: 'NON' is not a valid LinkedInCategory